In [3]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the md1
md1_path = r'D:\Jatin\College stuff\Major Project\Crowd\crowd-density-aspp\aspp_1_800.pt'
md1 = torch.load(md1_path, map_location=device).to(device)
md1.eval()

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Read video file
video_path = r'D:\Jatin\College stuff\Major Project\Crowd\crowd-density-aspp\263C044_060_c.mov'
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create VideoWriter object to write overlayed video
output_video_path = 'output_video_with_overlay.avi'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))

# Process each frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to PIL Image
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Apply transformation
    input_image = transform(frame_pil).unsqueeze(0).to(device)

    # Perform inference
    with torch.no_grad():
        output_density_map = md1(input_image)

    # Convert the output density map tensor to numpy array
    density_map_numpy = output_density_map.squeeze().cpu().numpy()

    # Resize density map to match frame size
    density_map_resized = cv2.resize(density_map_numpy, (frame_width, frame_height))

    # Normalize density map for visualization
    density_map_resized = cv2.normalize(density_map_resized, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Overlay density map on frame
    alpha = 0.5
    overlay = cv2.applyColorMap(density_map_resized, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(frame, 1, overlay, alpha, 0)

    # Write the frame with overlay to output video
    out.write(overlay)

    # Display progress
    frame_count += 1
    print(f'Processed frame {frame_count}/{total_frames}')

# Release resources
cap.release()
out.release()

print(f'Overlayed video saved at: {output_video_path}')


Processed frame 1/271
Processed frame 2/271
Processed frame 3/271
Processed frame 4/271
Processed frame 5/271
Processed frame 6/271
Processed frame 7/271
Processed frame 8/271
Processed frame 9/271
Processed frame 10/271
Processed frame 11/271
Processed frame 12/271
Processed frame 13/271
Processed frame 14/271
Processed frame 15/271
Processed frame 16/271
Processed frame 17/271
Processed frame 18/271
Processed frame 19/271
Processed frame 20/271
Processed frame 21/271
Processed frame 22/271
Processed frame 23/271
Processed frame 24/271
Processed frame 25/271
Processed frame 26/271
Processed frame 27/271
Processed frame 28/271
Processed frame 29/271
Processed frame 30/271
Processed frame 31/271
Processed frame 32/271
Processed frame 33/271
Processed frame 34/271
Processed frame 35/271
Processed frame 36/271
Processed frame 37/271
Processed frame 38/271
Processed frame 39/271
Processed frame 40/271
Processed frame 41/271
Processed frame 42/271
Processed frame 43/271
Processed frame 44/2

KeyboardInterrupt: 

In [4]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the model
md1_path = r'D:\Jatin\College stuff\Major Project\Crowd\crowd-density-aspp\Models\aspp\aspp_1_800.pt'
md1 = torch.load(md1_path, map_location=device).to(device)
md1.eval()

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Read video file
video_path = r'D:\Jatin\College stuff\Major Project\Crowd\crowd-density-aspp\263C044_060_c.mov'
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Set resize parameters
resize_width = 640  # Change this according to your preference
resize_height = 480  # Change this according to your preference

# Create window to display analyzed frames
cv2.namedWindow('Analyzed Frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Analyzed Frame', resize_width, resize_height)

# Process each frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (resize_width, resize_height))

    # Convert frame to PIL Image
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Apply transformation
    input_image = transform(frame_pil).unsqueeze(0).to(device)

    # Perform inference
    with torch.no_grad():
        output_density_map = md1(input_image)

    # Convert the output density map tensor to numpy array
    density_map_numpy = output_density_map.squeeze().cpu().numpy()

    # Resize density map to match frame size
    density_map_resized = cv2.resize(density_map_numpy, (resize_width, resize_height))

    # Normalize density map for visualization
    density_map_resized = cv2.normalize(density_map_resized, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Overlay density map on frame
    alpha = 0.5
    overlay = cv2.applyColorMap(density_map_resized, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(frame, 1, overlay, alpha, 0)

    # Display the analyzed frame
    cv2.imshow('Analyzed Frame', overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Display progress
    frame_count += 1
    print(f'Processed frame {frame_count}/{total_frames}')

# Release resources
cap.release()
cv2.destroyAllWindows()


Processed frame 1/271
Processed frame 2/271
Processed frame 3/271
Processed frame 4/271
Processed frame 5/271
Processed frame 6/271
Processed frame 7/271
Processed frame 8/271
Processed frame 9/271
Processed frame 10/271
Processed frame 11/271
Processed frame 12/271
Processed frame 13/271
Processed frame 14/271
Processed frame 15/271
Processed frame 16/271
Processed frame 17/271
Processed frame 18/271
Processed frame 19/271
Processed frame 20/271
Processed frame 21/271
Processed frame 22/271
Processed frame 23/271
Processed frame 24/271
Processed frame 25/271
Processed frame 26/271
Processed frame 27/271
Processed frame 28/271
Processed frame 29/271
Processed frame 30/271
Processed frame 31/271
Processed frame 32/271
Processed frame 33/271
Processed frame 34/271
Processed frame 35/271
Processed frame 36/271
Processed frame 37/271
Processed frame 38/271
Processed frame 39/271
Processed frame 40/271
Processed frame 41/271
Processed frame 42/271
Processed frame 43/271
Processed frame 44/2

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the model
md1_path = r'D:\Jatin\College stuff\Major Project\Crowd\crowd-density-aspp\Models\aspp\aspp_2_800.pt'
md1 = torch.load(md1_path, map_location=device).to(device)
md1.eval()

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Read video file
video_path = r'D:\Jatin\College stuff\Major Project\Data\263C044_060_c.mov'
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Set resize parameters
resize_width = 640  # Change this according to your preference
resize_height = 480  # Change this according to your preference

# Function to get bounding boxes
def get_bounding_boxes(density_map, num_boxes=5):
    # Threshold density map
    thresholded_map = cv2.threshold(density_map, 50, 255, cv2.THRESH_BINARY)[1]

    # Find contours
    contours, _ = cv2.findContours(thresholded_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:num_boxes]

    # Get bounding boxes
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]

    return bounding_boxes

# Create window to display analyzed frames
cv2.namedWindow('Analyzed Frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Analyzed Frame', resize_width, resize_height)

# Define the codec and create VideoWriter object for MP4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for MP4 format
output_video_path = r'D:\Jatin\College stuff\Major Project\Data\output_video.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (resize_width, resize_height))


# Process each frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (resize_width, resize_height))

    # Convert frame to PIL Image
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Apply transformation
    input_image = transform(frame_pil).unsqueeze(0).to(device)

    # Perform inference
    with torch.no_grad():
        output_density_map = md1(input_image)

    # Convert the output density map tensor to numpy array
    density_map_numpy = output_density_map.squeeze().cpu().numpy()

    # Resize density map to match frame size
    density_map_resized = cv2.resize(density_map_numpy, (resize_width, resize_height))

    # Normalize density map for visualization
    density_map_resized = cv2.normalize(density_map_resized, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Overlay density map on frame
    alpha = 0.5
    overlay = cv2.applyColorMap(density_map_resized, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(frame, 1, overlay, alpha, 0)

    # Get bounding boxes for high density areas
    bounding_boxes = get_bounding_boxes(density_map_resized)

    # Draw bounding boxes on overlay
    for x, y, w, h in bounding_boxes:
        cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Write frame to output video
    out.write(overlay)

    # Display the analyzed frame
    cv2.imshow('Analyzed Frame', overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Display progress
    frame_count += 1
    print(f'Processed frame {frame_count}/{total_frames}')

# Release resources
cap.release()
out.release()  # Release VideoWriter
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'model'